In [ ]:
# requirement for running this assignment 
# gym-duckietown --> env and maps 
# Tqdm
# Tensorboard

################# TODO ######################
# have to check the Tensorboard working with this implementation 
# Full running of the code from the notebook (locally)
# On colab

## Imitation Learning implementation

In [ ]:
# import 

import math
import numpy as np
import torch
from torch.utils.data import Dataset
import os
from typing import Tuple, List
from gym_duckietown.simulator import AGENT_SAFETY_RAD
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models.resnet import conv3x3

import numpy as np
from tqdm import tqdm
import cv2
import os

import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.tensorboard import SummaryWriter

from PIL import Image

from gym_duckietown.envs import DuckietownEnv
import argparse


### Dataset Buffer

In [ ]:
# Data util 

class MemoryMapDataset(Dataset):
    """Dataset to store multiple arrays on disk to avoid saturating the RAM"""

    def __init__(self, size: int, data_size: tuple, target_size: tuple, path: str):
        """
        Parameters
        ----------
        size : int
            Number of arrays to store, will be the first dimension of the resulting tensor dataset.
        data_size : tuple
            Size of data, may be 3D (CxHxW) for images or 2D/1D for features.
        target_size : tuple
            Size of the target, for our case it is a 1D array having, angular and linear speed.
        path : str
            Path where the file will be saved.
        """
        self.size = size
        self.data_size = data_size
        self.target_size = target_size
        self.path = path

        # Path for each array
        self.data_path = os.path.join(path, "data.dat")
        self.target_path = os.path.join(path, "target.dat")

        # Create arrays
        self.data = np.memmap(self.data_path, dtype="float32", mode="w+", shape=(self.size, *self.data_size))
        self.target = np.memmap(
            self.target_path, dtype="float32", mode="w+", shape=(self.size, *self.target_size)
        )

        # Initialize number of saved records to zero
        self.length = 0

        # keep track of real length in case of bypassing size value
        self.real_length = 0

    def __getitem__(self, item) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get one pair of training examples from the dataset.

        Parameters
        ----------
        item : int
            Index on the first dimension of the dataset.

        Returns
        -------
        sample, target : tuple
            Training sample consisting of data, label of data_size and target_size, respectively.
        """
        sample = torch.tensor(self.data[item, ...])
        target = torch.tensor(self.target[item, ...])

        return sample, target

    def __len__(self) -> int:
        """Get size (number of saved examples) of the dataset.

        Returns
        -------
        length : int
            Occupied length of the dataset. Note that it returns the number of saved examples rather than the maximum
            size used in the initialization.
        """
        return self.length

    def extend(self, observations: List[np.ndarray], actions: List[np.ndarray]):
        """Saves observations to the dataset. Iterates through the lists containing matching pairs of observations and
        actions. After saving each sample the dataset size is readjusted. If the dataset exceeds its maximum size
        it will start overwriting the firs experiences.

        Parameters
        ----------
        observations : List
            List containing np.ndarray observations of size data_size.
        actions
            List containing np.ndarray actions of size target_size.
        """
        for index, (observation, action) in enumerate(zip(observations, actions)):
            current_data_indx = self.real_length + index
            if self.real_length + index >= self.size:
                # it will be a circular by getting rid of old experiments
                current_data_indx %= self.size
            self.data[current_data_indx, ...] = observation.astype(np.float32)
            self.target[current_data_indx, ...] = action.astype(np.float32)
        if self.real_length >= self.size:
            self.length = self.size - 1
        else:
            self.length += len(observations)
        self.real_length += len(observations)

    def save(self):
        """In case of wanting to save the dataset this method should be implemented by flushing anc closing the memory
        map. Note that the files (depending on the size parameter) may occupy considerable amount of memory.
        """
        # TODO
        pass


### Interative Imitation Learning Class 

In [ ]:
class InteractiveImitationLearning:
    """
    A class used to contain main imitation learning algorithm
    ...
    Methods
    -------
    train(samples, debug)
        start training imitation learning
    """

    def __init__(self, env, teacher, learner, horizon, episodes, test=False):
        """
        Parameters
        ----------
        env :
            duckietown environment
        teacher :
            expert used to train imitation learning
        learner :
            model used to learn
        horizon : int
            which is the number of observations to be collected during one episode
        episode : int
            number of episodes which is the number of collected trajectories
        """

        self.environment = env
        self.teacher = teacher
        self.learner = learner
        self.test = test

        # from IIL
        self._horizon = horizon
        self._episodes = episodes

        # data
        self._observations = []
        self._expert_actions = []

        # statistics
        self.learner_action = None
        self.learner_uncertainty = None

        self.teacher_action = None
        self.active_policy = True  # if teacher is active

        # internal count
        self._current_horizon = 0
        self._episode = 0

        # event listeners
        self._episode_done_listeners = []
        self._found_obstacle = False
        # steering angle gain
        self.gain = 10

    def train(self, debug=False):
        """
        Parameters
        ----------
        teacher :
            expert used to train imitation learning
        learner :
            model used to learn
        horizon : int
            which is the number of observations to be collected during one episode
        episode : int
            number of episodes which is the number of collected trajectories
        """
        self._debug = debug
        for episode in range(self._episodes):
            self._episode = episode
            self._sampling()
            self._optimize()  # episodic learning
            self._on_episode_done()

    def _sampling(self):
        observation = self.environment.render_obs()
        for horizon in range(self._horizon):
            self._current_horizon = horizon
            action = self._act(observation)
            try:
                next_observation, reward, done, info = self.environment.step(
                    [action[0], action[1] * self.gain]
                )
            except Exception as e:
                print(e)
            if self._debug:
                self.environment.render()
            observation = next_observation

    # execute current control policy
    def _act(self, observation):
        if self._episode <= 1:  # initial policy equals expert's
            control_policy = self.teacher
        else:
            control_policy = self._mix()

        control_action = control_policy.predict(observation)

        self._query_expert(control_policy, control_action, observation)

        self.active_policy = control_policy == self.teacher
        if self.test:
            return self.learner_action

        return control_action

    def _query_expert(self, control_policy, control_action, observation):
        if control_policy == self.learner:
            self.learner_action = control_action
        else:
            self.learner_action = self.learner.predict(observation)

        if control_policy == self.teacher:
            self.teacher_action = control_action
        else:
            self.teacher_action = self.teacher.predict(observation)

        if self.teacher_action is not None:
            self._aggregate(observation, self.teacher_action)

        if self.teacher_action[0] < 0.1:
            self._found_obstacle = True
        else:
            self._found_obstacle = False

    def _mix(self):
        raise NotImplementedError()

    def _aggregate(self, observation, action):
        if not (self.test):
            self._observations.append(observation)
            self._expert_actions.append(action)

    def _optimize(self):
        if not (self.test):
            self.learner.optimize(self._observations, self._expert_actions, self._episode)
            print("saving model")
            self.learner.save()

    # TRAINING EVENTS

    # triggered after an episode of learning is done
    def on_episode_done(self, listener):
        self._episode_done_listeners.append(listener)

    def _on_episode_done(self):
        for listener in self._episode_done_listeners:
            listener.episode_done(self._episode)
        self.environment.reset()

#### Dagger Class

In [ ]:
"""
The Dagger Class is the important, which uses Interative Inmitation learnign 
* The interative imitation learning has most of the curcial functions implemented
* In my opinion we can move those functions to Dagger class.

"""

class DAgger(InteractiveImitationLearning):
    """
    DAgger algorithm to mix policies between learner and expert
    Ross, Stéphane, Geoffrey Gordon, and Drew Bagnell. "A reduction of imitation learning and structured prediction to no-regret online learning." Proceedings of the fourteenth international conference on artificial intelligence and statistics. 2011.
    ...
    Methods
    -------
    _mix
        used to return a policy teacher / expert based on random choice and safety checks
    """

    def __init__(self, env, teacher, learner, horizon, episodes, alpha=0.5, test=False):
        InteractiveImitationLearning.__init__(self, env, teacher, learner, horizon, episodes, test)
        # expert decay
        self.p = alpha
        self.alpha = self.p

        # thresholds used to give control back to learner once the teacher converges
        self.convergence_distance = 0.05
        self.convergence_angle = np.pi / 18

        # threshold on angle and distance from the lane when using the model to avoid going off track and env reset within an episode
        self.angle_limit = np.pi / 8
        self.distance_limit = 0.12

    def _mix(self):
        control_policy = np.random.choice(a=[self.teacher, self.learner], p=[self.alpha, 1.0 - self.alpha])
        if self._found_obstacle:
            return self.teacher
        try:
            lp = self.environment.get_lane_pos2(self.environment.cur_pos, self.environment.cur_angle)
        except:
            return control_policy
        if self.active_policy:
            # keep using tecaher untill duckiebot converges back on track
            if not (abs(lp.dist) < self.convergence_distance and abs(lp.angle_rad) < self.convergence_angle):
                return self.teacher
        else:
            # in case we are using our learner and it started to diverge a lot we need to give
            # control back to the expert
            if abs(lp.dist) > self.distance_limit or abs(lp.angle_rad) > self.angle_limit:
                return self.teacher
        return control_policy

    def _on_episode_done(self):
        self.alpha = self.p ** self._episode
        # Clear experience
        self._observations = []
        self._expert_actions = []

        InteractiveImitationLearning._on_episode_done(self)

### Teacher (Pure Pursuit policy)


In [ ]:


POSITION_THRESHOLD = 0.04
REF_VELOCITY = 0.7
FOLLOWING_DISTANCE = 0.24
AGENT_SAFETY_GAIN = 1.15


class PurePursuitPolicy:
    """
    A Pure Pusuit controller class to act as an expert to the model
    ...
    Methods
    -------
    forward(images)
        makes a model forward pass on input images

    loss(*args)
        takes images and target action to compute the loss function used in optimization

    predict(observation)
        takes an observation image and predicts using env information the action
    """

    def __init__(
        self, env, ref_velocity=REF_VELOCITY, following_distance=FOLLOWING_DISTANCE, max_iterations=1000
    ):
        """
        Parameters
        ----------
        ref_velocity : float
            duckiebot maximum velocity (default 0.7)
        following_distance : float
            distance used to follow the trajectory in pure pursuit (default 0.24)
        """
        self.env = env
        self.following_distance = following_distance
        self.max_iterations = max_iterations
        self.ref_velocity = ref_velocity

    def predict(self, observation):
        """
        Parameters
        ----------
        observation : image
            image of current observation from simulator
        Returns
        -------
        action: list
            action having velocity and omega of current observation
        """
        closest_point, closest_tangent = self.env.unwrapped.closest_curve_point(
            self.env.cur_pos, self.env.cur_angle
        )
        if closest_point is None or closest_tangent is None:
            self.env.reset()
            closest_point, closest_tangent = self.env.unwrapped.closest_curve_point(
                self.env.cur_pos, self.env.cur_angle
            )

        current_world_objects = self.env.objects
        # to slow down if there's a duckiebot in front of you
        # this is used to avoid hitting another moving duckiebot in the map
        # in case of training LFV baseline
        velocity_slow_down = 1
        for obj in current_world_objects:
            if not obj.static and obj.kind == "duckiebot":
                if True:
                    collision_penalty = abs(
                        obj.proximity(self.env.cur_pos, AGENT_SAFETY_RAD * AGENT_SAFETY_GAIN)
                    )
                    if collision_penalty > 0:
                        # this means we are approaching and we need to slow down
                        velocity_slow_down = collision_penalty
                        break

        lookup_distance = self.following_distance
        # projected_angle used to detect corners and to reduce the velocity accordingly
        projected_angle, _, _ = self._get_projected_angle_difference(0.3)
        velocity_scale = 1

        current_tile_pos = self.env.get_grid_coords(self.env.cur_pos)
        current_tile = self.env._get_tile(*current_tile_pos)
        if "curve" in current_tile["kind"] or abs(projected_angle) < 0.92:
            # slowing down by a scale of 0.5
            velocity_scale = 0.5
        _, closest_point, curve_point = self._get_projected_angle_difference(lookup_distance)

        if closest_point is None:  # if cannot find a curve point in max iterations
            return [0, 0]

        # Compute a normalized vector to the curve point
        point_vec = curve_point - self.env.cur_pos
        point_vec /= np.linalg.norm(point_vec)
        right_vec = np.array([math.sin(self.env.cur_angle), 0, math.cos(self.env.cur_angle)])
        dot = np.dot(right_vec, point_vec)
        omega = -1 * dot
        # range of dot is just -pi/2 and pi/2 and will be multiplied later by a gain adjustable if we are testing on a hardware or not
        velocity = self.ref_velocity * velocity_scale
        if velocity_slow_down < 0.2:
            velocity = 0
            omega = 0

        action = [velocity, omega]

        return action

    def _get_projected_angle_difference(self, lookup_distance):
        # Find the projection along the path
        closest_point, closest_tangent = self.env.closest_curve_point(self.env.cur_pos, self.env.cur_angle)

        iterations = 0
        curve_angle = None

        while iterations < 10:
            # Project a point ahead along the curve tangent,
            # then find the closest point to to that
            follow_point = closest_point + closest_tangent * lookup_distance
            curve_point, curve_angle = self.env.closest_curve_point(follow_point, self.env.cur_angle)

            # If we have a valid point on the curve, stop
            if curve_angle is not None and curve_point is not None:
                break

            iterations += 1
            lookup_distance *= 0.5

        if curve_angle is None:  # if cannot find a curve point in max iterations
            return None, None, None

        else:
            return np.dot(curve_angle, closest_tangent), closest_point, curve_point

### Model

In [ ]:
"""
The model learns from color image, but here we can try more variety

"""


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)


class BasicBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(inplanes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride

    def forward(self, x):
        conv1 = self.conv1(x)
        conv1 = self.bn1(conv1)
        conv1 = self.relu(conv1)

        conv2 = self.conv2(x)
        conv2 = self.bn2(conv2)

        return conv1 + conv2


class Dronet(nn.Module):
    """
    A class used to define action regressor model based on Dronet arch.
    Loquercio, Antonio, et al. "Dronet: Learning to fly by driving." IEEE Robotics and Automation Letters 3.2 (2018): 1088-1095.
    ...
    Methods
    -------
    forward(images)
        makes a model forward pass on input images
    loss(*args)
        takes images and target action to compute the loss function used in optimization
    predict(*args)
        takes images as input and predict the action space unnormalized
    """

    def __init__(self, num_outputs=2, max_velocity=0.7, max_steering=np.pi / 2):
        """
        Parameters
        ----------
        num_outputs : int
            number of outputs of the action space (default 2)
        max_velocity : float
            the maximum velocity used by the teacher (default 0.7)
        max_steering : float
            maximum steering angle as we are predicting normalized [0-1] (default pi/2)
        """
        super(Dronet, self).__init__()
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_outputs = num_outputs
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=2),
            nn.MaxPool2d(kernel_size=(3, 3), stride=[2, 2]),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            BasicBlock(32, 32, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            BasicBlock(32, 64, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            BasicBlock(64, 128, stride=2),
            Flatten(),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )

        self.num_feats_extracted = 2560
        # predicting steering angle
        self.steering_angle_channel = nn.Sequential(nn.Linear(self.num_feats_extracted, 1))

        # predicting if the bot should speed up or slow down
        self.speed_up_channel = nn.Sequential(nn.Linear(self.num_feats_extracted, 1))

        # Decaying speed up loss parameters
        self.decay = 1 / 10
        self.epoch_0 = 10
        self.epoch = 0

        # Max steering angle, minimum velocity and maximum velocity parameters
        self.max_steering = max_steering
        self.max_velocity = max_velocity
        self.max_velocity_tensor = torch.tensor(self.max_velocity).to(self._device)
        self.min_velocity = self.max_velocity * 0.5
        self.min_velocity_tensor = torch.tensor(self.min_velocity).to(self._device)

    def forward(self, images):
        """
        Parameters
        ----------
        images : tensor
            batch of input images to get normalized predicted action
        Returns
        -------
        action: tensor
            normalized predicted action from the model
        """
        features = self.feature_extractor(images)
        steering_angle = self.steering_angle_channel(features)
        is_speed_up = self.speed_up_channel(features)
        return is_speed_up, steering_angle

    def loss(self, *args):
        """
        Parameters
        ----------
        *args :
            takes batch of images and target action space to get the loss function.
        Returns
        -------
        loss: tensor
            loss function used by the optimizer to update the model
        """
        self.train()
        images, target = args
        is_speed_up, steering_angle = self.forward(images)
        criterion_v = nn.BCEWithLogitsLoss()
        speed_up = (
            (target[:, 0] > self.min_velocity).float().unsqueeze(1)
        )  # 0 for expert speeding up and 1 for slowing down for a corner or an incoming duckbot
        loss_steering_angle = F.mse_loss(steering_angle, target[:, 1].unsqueeze(1), reduction="mean")
        loss_v = criterion_v(is_speed_up, speed_up)
        loss = loss_steering_angle + loss_v * max(0, 1 - np.exp(self.decay * (self.epoch - self.epoch_0)))
        return loss

    def predict(self, *args):
        """
        Parameters
        ----------
        *args : tensor
            batch of input images to get unnormalized predicted action
        Returns
        -------
        action: tensor
            action having velocity and omega of shape (batch_size, 2)
        """
        images = args[0]
        is_speed_up, steering_angle = self.forward(images)
        is_speed_up = torch.sigmoid(is_speed_up)
        v_tensor = (is_speed_up) * self.max_velocity_tensor + (1 - is_speed_up) * self.min_velocity_tensor
        steering_angle = steering_angle * self.max_steering
        output = torch.cat((v_tensor, steering_angle), 1).squeeze().detach()
        return output.cpu().numpy()


if __name__ == "__main__":
    batch_size = 2
    img_size = (120, 160)
    model = Dronet()
    input_image = torch.rand((batch_size, 3, img_size[0], img_size[1])).to(model._device)
    prediction = model.predict(input_image)
    assert list(prediction.shape) == [batch_size, model.num_outputs]

### Neural Network Policy

In [ ]:



class NeuralNetworkPolicy:
    """
    A wrapper to train neural network model
    ...
    Methods
    -------
    optimize(observations, expert_actions, episode)
        train the model on the newly collected data from the simulator

    predict(observation)
        takes images and predicts the action space using the model

    save
        save a model checkpoint to storage location
    """

    def __init__(self, model, optimizer, storage_location, dataset, **kwargs):
        """
        Parameters
        ----------
        model :
            input pytorch model that will be trained
        optimizer :
            torch optimizer
        storage_location : string
            path of the model to be saved , the dataset and tensorboard logs
        dataset :
            object storing observation and labels from expert
        """
        self._train_iteration = 0
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Base parameters
        self.model = model.to(self._device)
        self.optimizer = optimizer
        self.storage_location = storage_location
        self.writer = SummaryWriter(self.storage_location)

        # Optional parameters
        self.epochs = kwargs.get("epochs", 10)
        self.batch_size = kwargs.get("batch_size", 32)
        self.input_shape = kwargs.get("input_shape", (60, 80))
        self.max_velocity = kwargs.get("max_velocity", 0.7)

        self.episode = 0

        self.dataset = dataset
        # Load previous weights
        if "model_path" in kwargs:
            self.model.load_state_dict(torch.load(kwargs.get("model_path"), map_location=self._device))
            print("Loaded ")

    def __del__(self):
        self.writer.close()

    def optimize(self, observations, expert_actions, episode):
        """
        Parameters
        ----------
        observations :
            input images collected from the simulator
        expert_actions :
            list of actions each action [velocity, steering_angle] from expert
        episode : int
            current episode number
        """
        print("Starting episode #", str(episode))
        self.episode = episode
        self.model.episode = episode
        # Transform newly received data
        observations, expert_actions = self._transform(observations, expert_actions)

        # Retrieve data loader
        dataloader = self._get_dataloader(observations, expert_actions)
        # Train model
        for epoch in tqdm(range(1, self.epochs + 1)):
            running_loss = 0.0
            self.model.epoch = epoch
            for i, data in enumerate(dataloader, 0):
                # Send data to device
                data = [variable.to(self._device) for variable in data]

                # zero the parameter gradients
                self.optimizer.zero_grad()

                # forward + backward + optimize
                loss = self.model.loss(*data)
                loss.backward()
                self.optimizer.step()

                # Statistics
                running_loss += loss.item()

            # Logging
            self._logging(loss=running_loss, epoch=epoch)

        # Post training routine
        self._on_optimization_end()

    def predict(self, observation):
        """
        Parameters
        ----------
        observations :
            input image from the simulator
        Returns
        ----------
        prediction:
            action space of input observation
        """
        # Apply transformations to data
        observation, _ = self._transform([observation], [0])
        observation = torch.tensor(observation)
        # Predict with model
        prediction = self.model.predict(observation.to(self._device))

        return prediction

    def save(self):
        torch.save(self.model.state_dict(), os.path.join(self.storage_location, "model.pt"))

    def _transform(self, observations, expert_actions):
        # Resize images
        observations = [
            Image.fromarray(cv2.resize(observation, dsize=self.input_shape[::-1]))
            for observation in observations
        ]
        # Transform to tensors
        compose_obs = Compose(
            [
                ToTensor(),
                Normalize(
                    (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
                ),  # using imagenet normalization values
            ]
        )

        observations = [compose_obs(observation).cpu().numpy() for observation in observations]
        try:
            # Scaling steering angle to become in range -1 to 1 to make it easier to regress
            # Scaling velocity to range 0-1 based on max velocity
            expert_actions = [
                np.array([expert_action[0] / self.max_velocity, expert_action[1] / (np.pi / 2)])
                for expert_action in expert_actions
            ]
        except:
            pass
        expert_actions = [torch.tensor(expert_action).cpu().numpy() for expert_action in expert_actions]

        return observations, expert_actions

    def _get_dataloader(self, observations, expert_actions):
        # Include new experiences
        self.dataset.extend(observations, expert_actions)
        dataloader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)
        return dataloader

    def _logging(self, **kwargs):
        epoch = kwargs.get("epoch")
        loss = kwargs.get("loss")
        self.writer.add_scalar("Loss/train/{}".format(self._train_iteration), loss, epoch)

    def _on_optimization_end(self):
        self._train_iteration += 1

### Main function

In [ ]:


def launch_env(map_name, randomize_maps_on_reset=False, domain_rand=False):
    environment = DuckietownEnv(
        domain_rand=domain_rand,
        max_steps=math.inf,
        map_name=map_name,
        randomize_maps_on_reset=False,
    )
    return environment


def teacher(env, max_velocity):
    return PurePursuitPolicy(env=env, ref_velocity=max_velocity)


def process_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--episode", "-i", default=10, type=int)
    parser.add_argument("--horizon", "-r", default=128, type=int)
    parser.add_argument("--learning-rate", "-l", default=2, type=int)
    parser.add_argument("--decay", "-d", default=2, type=int)
    parser.add_argument("--save-path", "-s", default="iil_baseline", type=str)
    parser.add_argument("--map-name", "-m", default="loop_empty", type=str)
    parser.add_argument("--num-outputs", "-n", default=2, type=int)
    parser.add_argument("--domain-rand", "-dr", action="store_true")
    parser.add_argument("--randomize-map", "-rm", action="store_true")
    return parser


if __name__ == "__main__":
    parser = process_args()
    input_shape = (120, 160)
    batch_size = 16
    epochs = 10
    learning_rates = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    # decays
    mixing_decays = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95]
    # Max velocity
    max_velocity = 0.5

    config = parser.parse_args()
    # check for  storage path
    if not (os.path.isdir(config.save_path)):
        os.makedirs(config.save_path)
    # launching environment
    environment = launch_env(
        config.map_name,
        domain_rand=config.domain_rand,
        randomize_maps_on_reset=config.randomize_map,
    )

    task_horizon = config.horizon
    task_episode = config.episode

    model = Dronet(num_outputs=config.num_outputs, max_velocity=max_velocity)
    policy_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rates[config.learning_rate])

    dataset = MemoryMapDataset(25000, (3, *input_shape), (2,), config.save_path)
    learner = NeuralNetworkPolicy(
        model=model,
        optimizer=policy_optimizer,
        storage_location=config.save_path,
        batch_size=batch_size,
        epochs=epochs,
        input_shape=input_shape,
        max_velocity=max_velocity,
        dataset=dataset,
    )

    algorithm = DAgger(
        env=environment,
        teacher=teacher(environment, max_velocity),
        learner=learner,
        horizon=task_horizon,
        episodes=task_episode,
        alpha=mixing_decays[config.decay],
    )

    algorithm.train(debug=True)  # DEBUG to show simulation

    environment.close()